In [1]:
# From ANdrej Karpathy's lecture "et's build GPT: from scratch, in code, spelled out."
# the input file is downloaded to the local folder

with open('input.txt','r',encoding='utf-8') as f:
    text=f.read()

In [2]:
print(text[:100])
print("length of dataset in characters:",len(text))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
length of dataset in characters: 1115394


getting list of characters in tiny shakepeare

In [8]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
# mapping chars to ingtegers for encoding and decoding
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode=lambda s:[stoi[c] for c in s]
decode=lambda l:''.join([itos[i] for i in l])

test1=encode("hii there")
out1=decode(test1)
print(test1)
print(out1)

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there
